In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.utils.data as data
import torchvision
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torchvision.models as models

import matplotlib.pyplot as plt
import numpy as np

import copy
from collections import namedtuple
import os,glob
import random
import time

import cv2 
from torch.utils.data import DataLoader, Dataset
from PIL import Image
from natsort import natsorted

from model.ResNet import Bottleneck, ResNet, ResNet50

device = torch.device('cuda:1' if torch.cuda.is_available() else 'cpu')
torch.cuda.is_available()

True

In [2]:
class ImageTransform() :
    def __init__(self) :
        self.data_transform = {
            'train' : transforms.Compose([
                # transforms.RandomResizedCrop(resize, scale=(0.5, 1.0)),
                # transforms.RandomHorizontalFlip(),
                transforms.ToTensor(),
                # transforms.Normalize(mean, std)
            ]),
            'val' : transforms.Compose([
                # transforms.Resize(256),
                # transforms.CenterCrop(resize),
                transforms.ToTensor(),
                # transforms.Normalize(mean, std)
            ])
        }
        
    def __call__(self, img, phase) :
        return self.data_transform[phase](img)

In [4]:
Model = ResNet50(num_classes=3, channels=1).to(device)
optimizer = optim.Adam(Model.parameters(), lr=1e-4)
criterion = nn.CrossEntropyLoss()
criterion = criterion.to(device)

In [3]:
batch_size = 16
red_dir = './new_data_175/red'
yellow_dir = './new_data_175/yellow'
green_dir = './new_data_175/green'

red_images_filepaths = sorted([os.path.join(red_dir, f) for f in os.listdir(red_dir)])
yellow_images_filepaths = sorted([os.path.join(yellow_dir, f) for f in os.listdir(yellow_dir)])
green_images_filepaths = sorted([os.path.join(green_dir, f) for f in os.listdir(green_dir)])

image_filepaths = [*red_images_filepaths, *yellow_images_filepaths, *green_images_filepaths]
correct_images_filepaths = [i for i in image_filepaths if cv2.imread(i) is not None]

random.seed(50)
random.shuffle(correct_images_filepaths)

train_images_filepaths = correct_images_filepaths[:420] # 140 * 3
val_images_filepaths = correct_images_filepaths[420:473] # 35 * 3
test_images_filepaths = correct_images_filepaths[473:]

print(len(train_images_filepaths), len(val_images_filepaths), len(test_images_filepaths))


420 53 52


In [6]:
## Eval Model

import pandas as pd
id_list = []
pred_list = []
_id = 0

model_dir = './experiment/'
best_model = torch.load(model_dir + natsorted(os.listdir(model_dir))[-1])
print('Model: {} loaded!'.format(natsorted(os.listdir(model_dir))[-1]))
Model.load_state_dict(best_model)

pred_color = 'None'

with torch.no_grad() : 
    for test_path in test_images_filepaths :
        img = Image.open(test_path)
        _id = label = test_path.split('/')[-2]
        transform = ImageTransform()
        img = transform(img, phase='val')
        img = img.unsqueeze(0)
        img = img.to(device)
        
        Model.eval()
        outputs = Model(img)
        # preds = F.softmax(outputs, dim=1)[:, 1].tolist()
        preds = F.softmax(outputs, dim=1)
        argmax_preds = np.argmax(preds.cpu()).tolist()
        
        if argmax_preds == 2 :
            pred_color = 'red'
        elif argmax_preds == 1 :
            pred_color = 'yellow'
        elif argmax_preds == 0 :
            pred_color = 'green'
            
        id_list.append(_id)
        pred_list.append(pred_color)
        
res = pd.DataFrame({
        'ground_truth' : id_list,
        'pred' : pred_list
})

res.sort_values(by='ground_truth', inplace=True)
res.reset_index(drop=True, inplace=True)

res.to_csv('./ResNet_eval.csv', index=False)
res.head(53)

/tmp/ipykernel_43039/2137764263.py:9: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  best_model = torch.load(model_dir + natsorted(os.listdir(model_dir))[-1])


Model: 0.906_2ResNet-model.pt loaded!


,ground_truth,pred
0,green,green
1,green,green
2,green,green
3,green,green
4,green,green
5,green,red
6,green,green
7,green,green
8,green,green
9,green,green
